<a href="https://colab.research.google.com/github/sarahwierzbicki/projects/blob/main/predict_cat_on_postings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [ ]:
!pip install PyPDF2
!pip install python-docx

pre-processing

In [ ]:
#imports
from PyPDF2 import PdfReader
from docx import Document
import spacy
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_csv('/content/filtered_jobpostings.csv')

In [ ]:
import preprocessing_module

In [ ]:
df

,Unnamed: 0,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available
0,70,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,0.0,46312.0,no
1,84,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,0.0,41048.0,no
2,85,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0,yes
3,101,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0,yes
4,102,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,0.0,97114.0,no
...,...,...,...,...,...,...,...,...,...,...,...
79294,123837,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes
79295,123843,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes
79296,123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0,yes
79297,123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,0.0,99201.0,no


In [ ]:
job_df = df.drop_duplicates()

In [ ]:
job_df['description'] = job_df['description'].str.strip()

In [ ]:
df['processed_description'] = df['description'].apply(preprocessing_module.process)

In [ ]:
df

,Unnamed: 0,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available,processed_description
0,70,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,0.0,46312.0,no,revaturerevature largest fastest growing emplo...
1,84,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,0.0,41048.0,no,galerie seeking experienced quality assurance ...
2,85,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0,yes,validation engineer labware limsfoster city va...
3,101,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0,yes,global financial services firm seeking experie...
4,102,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,0.0,97114.0,no,seasonal office admin responsibilities checkin...
...,...,...,...,...,...,...,...,...,...,...,...,...
79294,123837,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position clinical contracts analyst req locati...
79295,123843,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position quality engineer complaint investigat...
79296,123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0,yes,walnut creek office currently seeking attorney...
79297,123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,0.0,99201.0,no,company overview eps learning leading k supple...


predict

In [ ]:
from datasets import Dataset
from transformers import pipeline

In [ ]:
model_name = "/content/drive/MyDrive/capstone/roberta-for-resume-classifying-infer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
#make dataset
predict_df = df[['processed_description', 'title']]
predict_dataset = Dataset.from_pandas(predict_df)

In [ ]:
#tokenize inputs
def tokenize(batch):
      return tokenizer(batch['processed_description'], padding="max_length", truncation=True)

In [ ]:
predict_dataset = predict_dataset.remove_columns('title')

In [ ]:
dataset = predict_dataset.map(tokenize, batched = True)

Map:   0%|          | 0/79299 [00:00<?, ? examples/s]

In [ ]:
dataset.set_format("torch", columns=["input_ids", "attention_mask"])

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
test_args = TrainingArguments(
    output_dir="./results",
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=8,
    dataloader_drop_last=False
   )

In [ ]:
trainer = Trainer(
       model=model,
       args=test_args,
   )

In [ ]:
!pip install numpy==1.26.4

In [ ]:
results = trainer.predict(dataset)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sw21410n (sw21410n-pace-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
print(results.predictions)

[[-1.2369471  -0.7117753  -0.41041708 ... -0.19279218 -0.5962811
  -1.2038789 ]
 [-0.878607    0.05043141  3.3754227  ... -0.50795436 -0.23175752
  -0.8197447 ]
 [ 0.02365166  0.5447498   5.9596653  ...  0.01553252 -0.61523694
  -0.10430127]
 ...
 [-0.07640219  0.99222577  0.57192606 ... -0.90435594 -1.089117
  -1.1261909 ]
 [-1.0730764   1.1834569   1.823706   ...  0.07200671 -1.245295
   2.4091897 ]
 [-1.0619609  -0.3389496  -0.6497998  ...  0.71630687 -0.22698104
  -1.2410471 ]]


In [ ]:
print(results.metrics)

{'test_model_preparation_time': 0.0031, 'test_runtime': 509.2688, 'test_samples_per_second': 155.711, 'test_steps_per_second': 19.465}


In [ ]:
#manual mapping :,(
id2label = {0: "ACCOUNTANT", 1: 'ADVOCATE', 2: 'AGRICULTURE', 3: 'APPAREL', 4: 'ARTS', 5: 'AUTOMOBILE', 6: 'AVIATION', 7: 'BANKING', 8: 'BPO', 9: 'BUSNIESS-DEVELOPMENT', 10: 'CHEF', 11: 'CONSTRUCTION', 12: 'CONSULTANT', 13: 'DESIGNER', 14: 'DIGITAL-MEDIA', 15: 'ENGINEERING', 16: 'FINANCE', 17: 'FITNESS', 18: 'HEALTHCARE', 19: 'HR', 20: 'INFORMATION-TECHNOLOGY', 21: 'PUBLIC-RELATIONS', 22: 'SALES', 23: 'TEACHER'}
label2id = {"ACCOUNTANT": 0,'ADVOCATE': 1, 'AGRICULTURE': 2,  'APPAREL':3, 'ARTS': 4, 'AUTOMOBILE': 5, 'AVIATION': 6, 'BANKING': 7, 'BPO': 8, 'BUSNIESS-DEVELOPMENT': 9, 'CHEF': 10, 'CONSTRUCTION': 11, 'CONSULTANT': 12, 'DESIGNER': 13, 'DIGITAL-MEDIA': 14, 'ENGINEERING': 15, 'FINANCE': 16, 'FITNESS': 17, 'HEALTHCARE': 18, 'HR': 19, 'INFORMATION-TECHNOLOGY': 20, 'PUBLIC-RELATIONS': 21, 'SALES': 22, 'TEACHER': 23}

In [ ]:
predicted_indices = np.argmax(results.predictions, axis=1)

In [ ]:
prediction_df = pd.DataFrame(predicted_indices)

In [ ]:
prediction_df['Category'] = prediction_df[0].map(id2label)

In [ ]:
prediction_df

,0,Category
0,20,INFORMATION-TECHNOLOGY
1,10,CHEF
2,2,AGRICULTURE
3,7,BANKING
4,2,AGRICULTURE
...,...,...
79294,18,HEALTHCARE
79295,18,HEALTHCARE
79296,5,AUTOMOBILE
79297,4,ARTS


In [ ]:
df

,Unnamed: 0,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available,processed_description
0,70,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,0.0,46312.0,no,revaturerevature largest fastest growing emplo...
1,84,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,0.0,41048.0,no,galerie seeking experienced quality assurance ...
2,85,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0,yes,validation engineer labware limsfoster city va...
3,101,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0,yes,global financial services firm seeking experie...
4,102,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,0.0,97114.0,no,seasonal office admin responsibilities checkin...
...,...,...,...,...,...,...,...,...,...,...,...,...
79294,123837,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position clinical contracts analyst req locati...
79295,123843,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position quality engineer complaint investigat...
79296,123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0,yes,walnut creek office currently seeking attorney...
79297,123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,0.0,99201.0,no,company overview eps learning leading k supple...


In [ ]:
final_df = prediction_df.join(df)

In [ ]:
final_df.drop('Unnamed: 0', axis = 1, inplace=True)

In [ ]:
final_df

,0,Category,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available,processed_description
0,20,INFORMATION-TECHNOLOGY,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,0.0,46312.0,no,revaturerevature largest fastest growing emplo...
1,10,CHEF,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,0.0,41048.0,no,galerie seeking experienced quality assurance ...
2,2,AGRICULTURE,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0,yes,validation engineer labware limsfoster city va...
3,7,BANKING,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0,yes,global financial services firm seeking experie...
4,2,AGRICULTURE,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,0.0,97114.0,no,seasonal office admin responsibilities checkin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79294,18,HEALTHCARE,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position clinical contracts analyst req locati...
79295,18,HEALTHCARE,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes,position quality engineer complaint investigat...
79296,5,AUTOMOBILE,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0,yes,walnut creek office currently seeking attorney...
79297,4,ARTS,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,0.0,99201.0,no,company overview eps learning leading k supple...


In [ ]:
final_df[final_df['Category'] == 'DIGITAL-MEDIA']

,0,Category,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available,processed_description
12,14,DIGITAL-MEDIA,Ascendion,Quality Assurance Specialist,About Ascendion Ascendion is a full-service di...,"Seattle, WA",Full-time,Associate,FULL_TIME,66000.0,98101.0,yes,ascendion ascendion service digital engineerin...
68,14,DIGITAL-MEDIA,SKF Group,Manufacturing Systems Coordinator,Factory Digital Coordinator\n\nLocation: Sumte...,"Sumter, SC",Full-time,Entry level,FULL_TIME,0.0,29150.0,no,factory digital coordinator location sumter sc...
127,14,DIGITAL-MEDIA,Planet Interactive,Digital Designer,Planet Interactive is in search of a Digital D...,"Bolingbrook, IL",Contract,Associate,CONTRACT,78000.0,60440.0,yes,planet interactive search digital designer dev...
157,14,DIGITAL-MEDIA,SKF Group,VP of Digital Enablement and Technology,VP Digital Enablement and Technology Americas\...,"Blue Bell, PA",Full-time,Executive,FULL_TIME,0.0,19422.0,no,vp digital enablement technology americas loca...
164,14,DIGITAL-MEDIA,Brady Corporation,Digital Asset Management Specialist,Who we are:Brady makes products that make the ...,"Milwaukee, WI",Full-time,Entry level,FULL_TIME,0.0,53202.0,no,brady makes products world safer productive pl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79183,14,DIGITAL-MEDIA,Comcast,Xfinity Retail Sales Consultant,Comcast brings together the best in media and ...,"Olympia, WA",Full-time,Associate,FULL_TIME,33904.0,98501.0,yes,comcast brings best media technology drive inn...
79190,14,DIGITAL-MEDIA,Asmodee,Inside Sales Representative,Who We Are Asmodee is an entertainment leader ...,"Lino Lakes, MN",Full-time,Associate,FULL_TIME,0.0,56089.0,no,asmodee entertainment leader specialized board...
79258,14,DIGITAL-MEDIA,Asmodee,Trade Marketing Manager,Who We Are Asmodee is an entertainment leader ...,"Lino Lakes, MN",Full-time,Mid-Senior level,FULL_TIME,0.0,56089.0,no,asmodee entertainment leader specialized board...
79261,14,DIGITAL-MEDIA,Pinterest,Sr. Client Partner,About Pinterest:\n\nMillions of people across ...,"San Francisco, CA",Full-time,Mid-Senior level,FULL_TIME,0.0,94101.0,no,pinterest millions people world come pinterest...


In [ ]:
final_df.to_json('jobpostingswithcategory.json')

In [ ]:
pipe = pipeline(model = '/content/drive/MyDrive/capstone/roberta-for-resume-classifying-infer', task = 'text-classification')

Device set to use cuda:0


In [ ]:
pipe('passenger service concourse manager summary dedicated enthusiastic leader able motivate employees perform best providing excellent service developing ongoing profitable client relationships creative resourceful flexible able adapt changing priorities maintain positive attitude strong work ethic skills strategic planning team building personable client relationships dedicated flexible relationship building decision making time management conflict resolution adaptability ability work pressure creativity ability work team microsoft word word processing spreadsheet microsoft excel microsoft programs scheduling accomplishments promoted times years diverse skill set managing employees customers areas aviation business proven ability manage complex concourses atlanta airport established trustworthy relationship clients multiple concourses experience current passenger service concourse manager company city state maintain appearance equipment utilized day managing daily operation driving safety policies directly responsible overseeing zone leaders communicated trained pushed operational changes directed maintained healthy working environment working busy atmosphere maintained provided daily schedule dispatchers senior leaders attended delta shift briefing regularly effort communicate expectations day address possible concern raised delta staff addressed overall concourse operation concerns fulfill customer request regularly timely manner provide daily huddle direction staff preparation daily operation coach counsel employees recognize employees diffuse situations challenge delays maintain effective operational performance revamp processes necessary provide leadership employees daily provide hands team work focus drive dress code policy employee professionalism communicate daily client management key positioned employees involved daily operation drive customer satisfaction results cabin service manager company city state managed team man employees managed team leaders directly responsible building day schedule employees responsible coaching counseling documenting employees managed entire concourse ramp communicated managed dispatcher responsible providing dispatcher staff effort turn aircraft day time turn operation responsible managing processes cbp responsible controlling operation failures pushed safety policies regularly cabin service supervisor company city state assistant manager scheduling employees execute directives provided leadership support team leaders assistant aircraft needed continuously provide safety patrol ramp concourse daily held briefing communicate information pasted management train employees team leaders service aircraft cabin service supervisor company city state maintain appearance equipment utilized operations assistant manager scheduling employees execute directives provided leadership support team leaders assistant aircraft needed continuously provide safety patrol ramp concourse daily held briefing communicate information pasted management train employees team leaders service aircraft education training high school diploma academics college park high city state usa academics technical skills months experience cbp seal process admin office work compiling information applications typing submitting applications cbp managing approved rejected applicants handled weekly meeting cbp provided updated based meeting management maintained red black seal count list provided weekly seal list hr management directing employees coming office assistance professionally handling calls directing callers assisting senior managers office needs copying computer input uniform issuing sorting')

[{'label': 'AVIATION', 'score': 0.9780384302139282}]